In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize the input data
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convert labels to one-hot encoded format
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu')) 
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(train_images)

# Train the model
model.fit(datagen.flow(train_images, train_labels, batch_size=32),
          steps_per_epoch=len(train_images) // 32, epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

model.save('model/mnist_v5-2.h5')

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import glob
from keras.models import load_model


model = load_model('model/mnist_v5.h5')

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(28, 28), color_mode="grayscale")
    img_array = np.array(img).reshape(-1, 28, 28, 1) 
    img_array = img_array.astype('float32') / 255.0
    return img_array


# Preprocess all the images
image_files = glob.glob('data/*.JPG')  
images = np.vstack([preprocess_image(img_file) for img_file in image_files])

predictions = model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

# Print the filename and corresponding predicted class for each image
for img_file, pred_class in zip(image_files, predicted_classes):
    print(f"File: {img_file}, Predicted Class: {pred_class}")

2025-03-23 16:29:22.115451: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 16:29:22.132834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-23 16:29:22.152015: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-23 16:29:22.157231: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 16:29:22.173831: I tensorflow/core/platform/cpu_feature_guar

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
File: data/0.JPG, Predicted Class: 0
File: data/1.JPG, Predicted Class: 1
File: data/2.JPG, Predicted Class: 2
File: data/3.JPG, Predicted Class: 3
File: data/4.JPG, Predicted Class: 4
File: data/5.JPG, Predicted Class: 5
File: data/6.JPG, Predicted Class: 6
File: data/7.JPG, Predicted Class: 7
File: data/8.JPG, Predicted Class: 8
File: data/9.JPG, Predicted Class: 9
File: data/eight.JPG, Predicted Class: 8
File: data/five.JPG, Predicted Class: 5
File: data/four.JPG, Predicted Class: 4
File: data/nine.JPG, Predicted Class: 9
File: data/one.JPG, Predicted Class: 1
File: data/seven.JPG, Predicted Class: 7
File: data/six.JPG, Predicted Class: 6
File: data/three.JPG, Predicted Class: 3
File: data/two.JPG, Predicted Class: 2
File: data/zero.JPG, Predicted Class: 0


In [3]:
!pip install tf2onnx



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import tensorflow as tf

# 1) Load the .h5 model (disable compile if you only need inference)
model = tf.keras.models.load_model("model/mnist_v5.h5", compile=False)

# 2) Save it as a standard TensorFlow SavedModel
tf.saved_model.save(model, "model/mnist_v5_saved")

print("TF SavedModel exported to: model/mnist_v5_saved")


INFO:tensorflow:Assets written to: model/mnist_v5_saved/assets


INFO:tensorflow:Assets written to: model/mnist_v5_saved/assets


TF SavedModel exported to: model/mnist_v5_saved


In [8]:
# Convert the SavedModel to ONNX
!python -m tf2onnx.convert \
    --saved-model model/mnist_v5_saved \
    --output model/mnist_v5.onnx \
    --opset 13

<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2025-03-23 16:46:48,117 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2025-03-23 16:46:48,402 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2025-03-23 16:46:48,626 - INFO - Signatures found in model: [serving_default].
2025-03-23 16:46:48,627 - WARNING - '--signature_def' not specified, using first signature: serving_default
2025-03-23 16:46:48,627 - INFO - Output names: ['output_0']
I0000 00:00:1742748408.641973    1076 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1742748408.866797    1076 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-03-23 16:46:48,920 - 

In [10]:
"""
Example Python script that:
1) Loops through all images in data/*.JPG
2) Preprocesses each image (28×28 grayscale, normalized)
3) Sends them all in a single batch request to your OpenVINO Model Server endpoint
4) Prints each filename with its predicted class
"""

import glob
import numpy as np
import requests
from PIL import Image

# Your OVMS endpoint URL
ENDPOINT_URL = "https://mnist-netsentinel.apps.cloud.xtoph152.dfw.ocp.run/v2/models/mnist/infer"

def preprocess_image(image_path):
    # Convert to 28x28 grayscale, normalize to [0..1], and reshape
    img = Image.open(image_path).convert("L")
    img = img.resize((28, 28))
    arr = np.array(img, dtype=np.float32) / 255.0
    arr = arr.reshape(1, 28, 28, 1)
    return arr

def main():
    # Collect all JPG images
    image_files = glob.glob("data/*.JPG")
    if not image_files:
        print("No .JPG files found in data/ folder.")
        return

    # Preprocess each image and stack them
    all_images = [preprocess_image(img_file) for img_file in image_files]
    images = np.vstack(all_images)  # shape: (N, 28, 28, 1)
    batch_size = images.shape[0]

    # Flatten the pixel data to a Python list
    data_list = images.flatten().tolist()

    # Create the inference request payload (KFServing V2 protocol)
    # Adjust "name" if your model’s input tensor is named differently
    request_payload = {
        "inputs": [
            {
                "name": "inputs",
                "shape": [batch_size, 28, 28, 1],
                "datatype": "FP32",
                "data": data_list
            }
        ]
    }

    # Send POST request to OVMS
    response = requests.post(ENDPOINT_URL, json=request_payload, verify=False)
    if response.status_code != 200:
        print(f"Request failed with status code {response.status_code}\nResponse: {response.text}")
        return

    # Parse the response JSON
    resp_json = response.json()
    # Typically the output shape will be (N, 10) for MNIST
    output_data = resp_json["outputs"][0]["data"]
    output_array = np.array(output_data).reshape(batch_size, -1)

    # Argmax over the last dimension to get the digit classes
    predicted_classes = np.argmax(output_array, axis=1)

    # Print each filename alongside its predicted class
    for img_file, pred_class in zip(image_files, predicted_classes):
        print(f"File: {img_file}, Predicted Class: {pred_class}")

if __name__ == "__main__":
    main()


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mnist-netsentinel.apps.cloud.xtoph152.dfw.ocp.run'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


File: data/0.JPG, Predicted Class: 0
File: data/1.JPG, Predicted Class: 1
File: data/2.JPG, Predicted Class: 2
File: data/3.JPG, Predicted Class: 3
File: data/4.JPG, Predicted Class: 4
File: data/5.JPG, Predicted Class: 5
File: data/6.JPG, Predicted Class: 6
File: data/7.JPG, Predicted Class: 7
File: data/8.JPG, Predicted Class: 8
File: data/9.JPG, Predicted Class: 9
File: data/eight.JPG, Predicted Class: 8
File: data/five.JPG, Predicted Class: 5
File: data/four.JPG, Predicted Class: 4
File: data/nine.JPG, Predicted Class: 9
File: data/one.JPG, Predicted Class: 1
File: data/seven.JPG, Predicted Class: 7
File: data/six.JPG, Predicted Class: 6
File: data/three.JPG, Predicted Class: 3
File: data/two.JPG, Predicted Class: 2
File: data/zero.JPG, Predicted Class: 0
